In [1]:
!pip install ipython-sql


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import sqlite3

In [12]:
# Load two CSV files into DataFrames
df_menu_items = pd.read_csv('menu_items.csv')  # First CSV file
df_order_details = pd.read_csv('order_details.csv')  # Second CSV file

print(df_menu_items.head())
print(df_order_details.head())

   menu_item_id      item_name  category  price
0           101      Hamburger  American  12.95
1           102   Cheeseburger  American  13.95
2           103        Hot Dog  American   9.00
3           104  Veggie Burger  American  10.50
4           105   Mac & Cheese  American   7.00
   order_details_id  order_id order_date   order_time  item_id
0                 1         1     1/1/23  11:38:36 AM    109.0
1                 2         2     1/1/23  11:57:40 AM    108.0
2                 3         2     1/1/23  11:57:40 AM    124.0
3                 4         2     1/1/23  11:57:40 AM    117.0
4                 5         2     1/1/23  11:57:40 AM    129.0


In [13]:
# Create a connection to a sqlite database (It will create one if it doesn't exist)
conn = sqlite3.connect("example_3.db")

In [14]:
# Load DataFrames into SQL tables
df_menu_items.to_sql('menu_items', conn, if_exists='replace', index=False)
df_order_details.to_sql('order_details', conn, if_exists='replace', index=False)

12234

In [15]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [31]:
 %reload_ext sql

In [21]:
%sql sqlite:///example_3.db

## Combine the menu_items and order_details tables into a single table.

In [30]:
%%sql
SELECT * FROM menu_items LIMIT 10;

SELECT * FROM order_details LIMIT 10;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.
Done.


order_details_id,order_id,order_date,order_time,item_id
1,1,1/1/23,11:38:36 AM,109.0
2,2,1/1/23,11:57:40 AM,108.0
3,2,1/1/23,11:57:40 AM,124.0
4,2,1/1/23,11:57:40 AM,117.0
5,2,1/1/23,11:57:40 AM,129.0
6,2,1/1/23,11:57:40 AM,106.0
7,3,1/1/23,12:12:28 PM,117.0
8,3,1/1/23,12:12:28 PM,119.0
9,4,1/1/23,12:16:31 PM,117.0
10,5,1/1/23,12:21:30 PM,117.0


In [35]:
%%sql

SELECT *
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id LIMIT 20;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


order_details_id,order_id,order_date,order_time,item_id,menu_item_id,item_name,category,price
1,1,1/1/23,11:38:36 AM,109.0,109,Korean Beef Bowl,Asian,17.95
2,2,1/1/23,11:57:40 AM,108.0,108,Tofu Pad Thai,Asian,14.5
3,2,1/1/23,11:57:40 AM,124.0,124,Spaghetti,Italian,14.5
4,2,1/1/23,11:57:40 AM,117.0,117,Chicken Burrito,Mexican,12.95
5,2,1/1/23,11:57:40 AM,129.0,129,Mushroom Ravioli,Italian,15.5
6,2,1/1/23,11:57:40 AM,106.0,106,French Fries,American,7.0
7,3,1/1/23,12:12:28 PM,117.0,117,Chicken Burrito,Mexican,12.95
8,3,1/1/23,12:12:28 PM,119.0,119,Chicken Torta,Mexican,11.95
9,4,1/1/23,12:16:31 PM,117.0,117,Chicken Burrito,Mexican,12.95
10,5,1/1/23,12:21:30 PM,117.0,117,Chicken Burrito,Mexican,12.95


## What were the least and most ordered items? What categories were they in?

In [38]:
%%sql

SELECT item_name, COUNT(order_details_id) as num_purchases
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id
    GROUP BY item_name;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


item_name,num_purchases
None,137
California Roll,355
Cheese Lasagna,207
Cheese Quesadillas,233
Cheeseburger,583
Chicken Burrito,455
Chicken Parmesan,364
Chicken Tacos,123
Chicken Torta,379
Chips & Guacamole,237


#### Least ordered item on the menu

In [50]:
%%sql

SELECT item_name, category, COUNT(order_details_id) AS num_purchases
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id
    GROUP BY item_name, category
    ORDER BY num_purchases LIMIT 1;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


item_name,category,num_purchases
Chicken Tacos,Mexican,123


#### Most ordered item on the menu

In [49]:
%%sql

SELECT item_name, category, COUNT(order_details_id) AS num_purchases
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id
    GROUP BY item_name, category
    ORDER BY num_purchases DESC LIMIT 1;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


item_name,category,num_purchases
Hamburger,American,622


## Combined output

In [47]:
%%sql
WITH item_counts AS (
    SELECT 
        mi.item_name, 
        mi.category,  -- Include category
        COUNT(od.order_details_id) AS num_purchases
    FROM order_details od
    LEFT JOIN menu_items mi ON od.item_id = mi.menu_item_id
    GROUP BY mi.item_name, mi.category
)

-- Get most and least ordered items
SELECT * FROM item_counts
WHERE num_purchases = (SELECT MAX(num_purchases) FROM item_counts)
   OR num_purchases = (SELECT MIN(num_purchases) FROM item_counts);


   sqlite:///example.db
 * sqlite:///example_3.db
Done.


item_name,category,num_purchases
Chicken Tacos,Mexican,123
Hamburger,American,622


## What were the top 5 orders that spent the most money?

In [55]:
%%sql

SELECT order_id, ROUND(SUM(price), 2) AS total_spend
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id 
    GROUP BY order_id
    LIMIT 20;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


order_id,total_spend
1,17.95
2,64.45
3,24.9
4,12.95
5,12.95
6,21.95
7,9.0
8,9.0
9,132.25
10,22.5


### These are the Top 5 

In [60]:
%%sql

SELECT order_id, item_name, ROUND(SUM(price), 2) AS total_spend
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id 
    GROUP BY order_id
    ORDER BY total_spend DESC LIMIT 5;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


order_id,item_name,total_spend
440,Steak Tacos,192.15
2075,Orange Chicken,191.05
1957,Orange Chicken,190.1
330,Orange Chicken,189.7
2675,Hamburger,185.1


Order that has a highest spend has an Order ID **440**.

## View the details of the highest spend order. What insights can you gather from the Data?

In [66]:
%%sql

SELECT *
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id WHERE order_id = 440;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


order_details_id,order_id,order_date,order_time,item_id,menu_item_id,item_name,category,price
1003,440,1/8/23,12:16:34 PM,116.0,116,Steak Tacos,Mexican,13.95
1004,440,1/8/23,12:16:34 PM,103.0,103,Hot Dog,American,9.0
1005,440,1/8/23,12:16:34 PM,124.0,124,Spaghetti,Italian,14.5
1006,440,1/8/23,12:16:34 PM,125.0,125,Spaghetti & Meatballs,Italian,17.95
1007,440,1/8/23,12:16:34 PM,125.0,125,Spaghetti & Meatballs,Italian,17.95
1008,440,1/8/23,12:16:34 PM,126.0,126,Fettuccine Alfredo,Italian,14.5
1009,440,1/8/23,12:16:34 PM,126.0,126,Fettuccine Alfredo,Italian,14.5
1010,440,1/8/23,12:16:34 PM,109.0,109,Korean Beef Bowl,Asian,17.95
1011,440,1/8/23,12:16:34 PM,127.0,127,Meat Lasagna,Italian,17.95
1012,440,1/8/23,12:16:34 PM,113.0,113,Edamame,Asian,5.0


In [67]:
%%sql

SELECT category, COUNT(item_id) AS num_items
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id WHERE order_id = 440
    GROUP BY category;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


category,num_items
American,2
Asian,2
Italian,8
Mexican,2


### This is from the Top 5

In [68]:
%%sql

SELECT category, COUNT(item_id) AS num_items
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id WHERE order_id IN (440, 2075, 1957, 330, 2675)
    GROUP BY category;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


category,num_items
American,10
Asian,17
Italian,26
Mexican,16


In [69]:
%%sql

SELECT order_id, category, COUNT(item_id) AS num_items
FROM order_details od LEFT JOIN menu_items mi
    ON od.item_id = mi.menu_item_id 
    WHERE order_id IN (440, 2075, 1957, 330, 2675)
    GROUP BY order_id, category;

   sqlite:///example.db
 * sqlite:///example_3.db
Done.


order_id,category,num_items
330,American,1
330,Asian,6
330,Italian,3
330,Mexican,4
440,American,2
440,Asian,2
440,Italian,8
440,Mexican,2
1957,American,3
1957,Asian,3
